<a href="https://colab.research.google.com/github/Bae-hong-seob/University-3-2/blob/main/Big_data_process_and_application/5_%EC%84%A0%ED%98%95%ED%9A%8C%EA%B7%80%EC%8B%A4%EC%8A%B5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pyspark
!apt install openjdk-8-jdk-headless -qql

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.4 MB 44 kB/s 
     |████████████████████████████████| 199 kB 42.6 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845513 sha256=879ba05c254cdfcab8ec7b4be98fe334971846ce7e9ba04b777f764ff90222eb
  Stored in directory: /root/.cache/pip/wheels/42/59/f5/79a5bf931714dcd201b26025347785f087370a10a3329a899c
Successfully built pyspark
E: Command line option 'l' [from -qql] is not understood in combination with the other options.


In [ ]:
import pyspark
from pyspark.sql import *

In [ ]:
spark = SparkSession.builder.getOrCreate()
from pyspark.sql.types import *

In [ ]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression

#inferSchema : 데이터의 스키마를 찾아주는 옵션
dataset = spark.read.csv("/content/drive/MyDrive/3-2 university/Big_data_process_and_application/input/boston.csv", inferSchema=True, header=True)

In [ ]:
#모든 피처를 하나의 벡터 열에 입력
assembler = VectorAssembler(inputCols=["CRIM", "ZN","INDUS","CHAS","NOX","RM",
                                       "AGE","DIS","RAD","TAX","PTRATIO","B","LSTAT"],
                            outputCol = "Attributes")

output = assembler.transform(dataset)

#input, output
finalized_data = output.select("Attributes", "MEDV")

finalized_data.show()

+--------------------+----+
|          Attributes|MEDV|
+--------------------+----+
|[0.00632,18.0,2.3...|24.0|
|[0.02731,0.0,7.07...|21.6|
|[0.02729,0.0,7.07...|34.7|
|[0.03237,0.0,2.18...|33.4|
|[0.06905,0.0,2.18...|36.2|
|[0.02985,0.0,2.18...|28.7|
|[0.08829,12.5,7.8...|22.9|
|[0.14455,12.5,7.8...|27.1|
|[0.21124,12.5,7.8...|16.5|
|[0.17004,12.5,7.8...|18.9|
|[0.22489,12.5,7.8...|15.0|
|[0.11747,12.5,7.8...|18.9|
|[0.09378,12.5,7.8...|21.7|
|[0.62976,0.0,8.14...|20.4|
|[0.63796,0.0,8.14...|18.2|
|[0.62739,0.0,8.14...|19.9|
|[1.05393,0.0,8.14...|23.1|
|[0.7842,0.0,8.14,...|17.5|
|[0.80271,0.0,8.14...|20.2|
|[0.7258,0.0,8.14,...|18.2|
+--------------------+----+
only showing top 20 rows



In [ ]:
#8:2로 train_data, test_data split
train_data, test_data = finalized_data.randomSplit([0.8, 0.2])

regressor = LinearRegression(featuresCol = "Attributes", labelCol = "MEDV")

#train_data set 학습
regressor = regressor.fit(train_data)

#test_data set 예측
pred = regressor.evaluate(test_data)

#모델 예측
pred.predictions.show()

+--------------------+----+------------------+
|          Attributes|MEDV|        prediction|
+--------------------+----+------------------+
|[0.01096,55.0,2.2...|22.0| 27.82323837668836|
|[0.01951,17.5,1.3...|33.0|23.348356214185756|
|[0.02009,95.0,2.6...|50.0|   43.795087275146|
|[0.02985,0.0,2.18...|28.7|25.145950151002484|
|[0.03445,82.5,2.0...|24.1| 30.03546904967881|
|[0.03615,80.0,4.9...|27.9| 32.07362659391295|
|[0.03768,80.0,1.5...|34.6| 35.49421860623207|
|[0.03961,0.0,5.19...|21.1| 20.23758385528349|
|[0.04011,80.0,1.5...|33.3| 36.87621271272494|
|[0.04301,80.0,1.9...|18.2|14.434375688867455|
|[0.04379,80.0,3.3...|19.4|26.124580458852698|
|[0.0459,52.5,5.32...|22.3| 27.54385172577473|
|[0.05302,0.0,3.41...|28.7|30.805674022874197|
|[0.0536,21.0,5.64...|25.0| 27.60494789337492|
|[0.05644,40.0,6.4...|32.4|35.386816290762894|
|[0.05646,0.0,12.8...|21.2| 21.21217038063563|
|[0.0578,0.0,2.46,...|37.2|32.842653412179246|
|[0.06151,0.0,5.19...|18.7|21.194522513363086|
|[0.06466,70.

In [ ]:
#회귀 모형의 계수
coeff = regressor.coefficients

#X, Y 절편
intr = regressor.intercept

print("The coefficient of the model is : %a" %coeff)
print("The intercept of the model is : %f" %intr)

The coefficient of the model is : DenseVector([-0.1231, 0.0534, 0.0177, 1.8703, -18.4318, 3.7448, 0.0057, -1.4893, 0.2903, -0.012, -1.0213, 0.0073, -0.521])
The intercept of the model is : 38.875134


In [ ]:
from pyspark.ml.evaluation import RegressionEvaluator
eval = RegressionEvaluator(labelCol = "MEDV", predictionCol='prediction', metricName='rmse')

#Root Mean Square Error
rmse = eval.evaluate(pred.predictions)
print("RMSE : %.3f" %rmse)

#Mean Square Error
mse = eval.evaluate(pred.predictions, {eval.metricName : "mse"})
print("MSE : %.3f" %mse)

#Mean Absolute Error
mae = eval.evaluate(pred.predictions, {eval.metricName : "mae"})
print("MAE : %.3f" %mae)

#r2 - coefficient of determination
r2 = eval.evaluate(pred.predictions, {eval.metricName : "r2"})
print("r2 : %.3f" %r2)

RMSE : 5.183
MSE : 26.862
MAE : 3.626
r2 : 0.678
